# Facial Emotion Detection using CNN and VGG16
This notebook contains two deep learning models:
1. A Convolutional Neural Network (CNN) model (your existing model).
2. A VGG16-based model using transfer learning.

Both models will be trained on the same dataset and evaluated for performance.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TensorFlow version:", tf.__version__)

## Data Preparation
Update the `train_dir` and `val_dir` variables to point to your training and validation data directories. The images will be resized to `224x224` pixels (the required input size for VGG16).

In [ ]:
# Parameters and Data Setup
train_dir = 'data/train'  # Update to your training data path
val_dir = 'data/val'      # Update to your validation data path

img_height, img_width = 224, 224
batch_size = 32
num_classes = 7

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

## VGG16-Based Model Definition

In [ ]:
input_shape = (img_height, img_width, 3)

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False

# Add custom layers
vgg16_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
vgg16_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

vgg16_model.summary()

## Train the VGG16 Model

In [ ]:
epochs = 10
history_vgg16 = vgg16_model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)
print("Training complete.")

## Save the VGG16 Model

In [ ]:
vgg16_model.save('vgg16_emotion_model.h5')
print("Model saved as vgg16_emotion_model.h5")